# Topic Modelling in Feedbacks Domain Using Latent Dirichlet Allocation



Partner: Michael Hewlett, Canada Digital Analytics Team

Mentor: Jungyeul Park, the University of British Columbia (UBC)

Author: Alex Chen, Student at the UBC Master of Data Science Computational Linguistics Program

Version: 2021.06.26




## Overview

This notebook applies unsupervised learning on tens of thousands comments to build a topic model and discover emerging clusters/topics that might deserve new tags of their own. The model also generates candidate new tags for domain experts to use. 

## Set-up

### Declare Constant Variables


In [1]:
 
 id_file_path = 'ids.csv'
csv_file_path = '../../../data/vaccine_full.csv'


### Trim Excessive Printout

In [2]:
from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')

### Installs

In [3]:
!pip install pyLDAvis==3.2.2
!pip install gensim

clear_output()

### Imports

In [4]:
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
import spacy
nlp = spacy.load('en')
from spacy.tokenizer import Tokenizer
tokenizer = Tokenizer(nlp.vocab)
import nltk
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import gensim
from gensim.parsing.preprocessing import strip_punctuation
from gensim.parsing.preprocessing import remove_stopwords
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from gensim import models
import pyLDAvis.gensim
clear_output()

## Load Data

In [5]:
df = pd.read_csv(csv_file_path)
cols = ['Unique ID', 'Comment', 'Tags']
df = df[cols]
df.columns = ['id', 'comment', 'tag']
display(df.head(0))
print()
df.shape


,id,comment,tag


(29292, 3)

In [7]:
iddf = pd.read_csv(id_file_path)
merged = pd.merge(iddf,df,on='id')

In [8]:
 columns = ['comment', 'tag']
 df = merged[columns]
 

In [9]:
docs = df['comment'].tolist()


## Build a topic model

In [10]:
def show_pyldavis(docs, num_topics):

#docs is a list of strings
#num_topics for the LDA model

  docs = [remove_stopwords(doc.lower()) for doc in docs]

  token_ = [strip_punctuation(' '.join([str(x) for x in nlp(doc)])) for doc in docs]

  token_ = [x.split(" ") for x in token_]

  lmtzr = WordNetLemmatizer()

  for token in token_:
      token = [lmtzr.lemmatize(x) for x in token]
      token = [x for x in token if x not in set(stopwords.words('english'))]

  bigram = Phrases(token_, min_count=5, threshold=2,delimiter=b' ')

  bigram_phraser = Phraser(bigram)

  bigram_token = []
  for sent in token_:
      bigram_token.append(bigram_phraser[sent])

  #now you can make dictionary of bigram token 
  dictionary = gensim.corpora.Dictionary(bigram_token)

  corpus = [dictionary.doc2bow(text) for text in bigram_token]

  lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, passes=1)

  viz = pyLDAvis.gensim.prepare(lda, corpus, dictionary)

  return (viz,lda, dictionary, corpus)
clear_output()

In [11]:
viz,lda, dictionary, corpus = show_pyldavis(docs, 11)
clear_output()

In [12]:
dominant_topics = []
for doc in corpus:
    row = lda.get_document_topics(bow=doc)
    dom_topic = sorted(row, key = lambda x: x[1], reverse=True)[0][0]
    dominant_topics.append(dom_topic)

In [13]:
df['topic'] = dominant_topics

In [14]:
pyLDAvis.save_html(viz, 'topic_model_visualization.html')

clear_output()

pyLDAvis.enable_notebook()
viz

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.142604  0.162744       1        1  11.180178
5     -0.065669 -0.000382       2        1  10.607656
8      0.130480  0.159046       3        1   9.935322
4     -0.124703  0.023648       4        1   9.177169
9     -0.073612  0.090989       5        1   9.141120
7      0.237790 -0.003647       6        1   8.952752
2      0.040442 -0.039720       7        1   8.865328
6     -0.094761 -0.221001       8        1   8.605080
1      0.030586 -0.157670       9        1   8.349533
10     0.078611 -0.018920      10        1   7.779407
0     -0.016561  0.004911      11        1   7.406455, topic_info=               Term         Freq        Total Category  logprob  loglift
4           vaccine  6714.000000  6714.000000  Default  30.0000  30.0000
94         vaccines  1427.000000  1427.000000  Default  29.0000  29.0000
841   covid vaccine   972.000000   972.000000  Default  28.0000  28.0000
133        covid 19  1080.000000  1080.000000  Default  27.0000  27.0000
202         effects   889.000000   889.000000  Default  26.0000  26.0000
...             ...          ...          ...      ...      ...      ...
498            tell    49.799583    89.331784  Topic11  -5.3888   2.0185
332            like    63.469987   423.970013  Topic11  -5.1462   0.7037
3005           live    56.845849   335.792274  Topic11  -5.2565   0.8267
65      astrazeneca    51.188789   368.423873  Topic11  -5.3613   0.6291
105          people    50.256169   943.332522  Topic11  -5.3797  -0.3295

[653 rows x 6 columns], token_table=      Topic      Freq Term
term                      
810       2  0.196566    1
810       4  0.021841    1
810       5  0.014560    1
810       6  0.007280    1
810       7  0.189286    1
...     ...       ...  ...
1596      6  0.797546   ’s
1596      9  0.010494   ’s
1596     10  0.188892   ’s
413       6  0.985100    “
414       6  0.991035    ”

[1410 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 6, 9, 5, 10, 8, 3, 7, 2, 11, 1])

## Next steps

Now that we have machine-generated topic and expert tag pinned against each other:

In [15]:
df.head(0)

,comment,tag,topic


The user may use their favorate data analysis tool (e.g. Excel) to find out which topic consists of new information that the domain expert may find interesting.

In [16]:
topic_of_interest = 0


Once the user decides which topic is of interest, they can use the following line to get top 10 terms in the topic. These terms can be used as candidates or parts of the new tag.

In [17]:
[dictionary[tpl[0]] for tpl in lda.get_topic_terms(topic_of_interest)]

['want know',
 'work',
 'need know',
 'vaccine',
 'virus',
 'vaccinated',
 'covid',
 'ingredients',
 'canada',
 'trying']


Reference: 

[vtonmail's notebook on kaggle.com](https://www.kaggle.com/vtonmail/pyldavis-on-employee-reviews)